In [45]:
import numpy as np 
import pandas as pd 

                                                        Методы решения уравнений переноса
Найти аналитическое и численное решение смешанной задачи для уравнения переноса в квадрате $0 \le x, t \le 1$ и сравнить их значения в одиннадцати 
равноудаленных точках в момент времени $t = 1$ 



      Задание 16.
Дифференциальная задача:
\begin{equation*}
 \begin{cases}
    \frac{du}{dt} - 6\frac{du}{dx} = 0, 0 < t \le 1, 0 \le x < 1 \\
    u(x, 0) = ln(1 + x), u(1, t) = ln(2 + 6t)    
 \end{cases}
\end{equation*}
Разностная схема:
$$D_h = {(x_l, t^n) : x_l = lh, hL = 1, l = 0,...,L; t^n = n \tau, \tau N = 1, n = 0,..., N}$$

\begin{equation*}
 \begin{cases}
      u_l^{n+1} = u_l^n + \frac{\tau}{h}(2u_{l+3}^n - 9u_{l+2}^n + 18u_{l+1}^n - 11u_l^n) + \\
                + \frac{18\tau^2}{h^2}(-u_{l+3}^n + 4u_{l+2}^n - 5u_{l+1}^n + 2u_l^n) + \\
                + \frac{36\tau^3}{h^3}(u_{l+3}^n - 3u_{l+2}^n + 3u_{l+1}^n - u_l^n), l = 0,...,L-3; n = 0,...,N-1 \\
      u_l^0 = ln(1 + x_l), l = 0,...,L; \\
      u_L^n = ln(2 + 6t^n), n = 1,...,N \\
      u_{L-1}^n = ?, n = 1,...,N; \\ 
      u_{L-2}^n = ?, n = 1,...,N  
 \end{cases}
\end{equation*}


        Аналитическое решение 
Первые интеграллы:
$$dt = - \frac{dx}{6}$$
$$du = 0$$
Следовательно их решения:
$$v_1(x, t) = t + \frac{x}{6} = const$$
$$v_2(x, t, u) = u = const$$
Соответственно в $t = 0$:
$$x = 6v_{10}$$
$$u = v_{20}$$
Подставив $x, u$ в первое начальное условие получаем:
$$u = ln(1 + 6(t + \frac{x}{6}))$$
Соответственно в $x = 1$:
$$t = v_{11} - 1/6$$
$$u = v_{21}$$
Подставив $x, u$ в первое начальное условие получаем:
$$u = ln(2 + 6(t + \frac{x}{6} - 1/6))$$
Тогда аналитическое решение выглядит, вот так:
$$u = ln(1 + 6(t + \frac{x}{6}))$$

In [46]:

def analitical_solution(x, t):
    return np.log(1 + 6 * t + x)

        Численное решение

In [47]:
def grid(L, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, 1, N)

#начальные условия
def phi(x):
    return np.log(1+x)

def psi(t):
    return np.log(2 + 6 * t)

def u_x(t):
    return -(2 + 6 * t)**-1

def u_xx(t):
    return -(2 + 6 * t)**-2

def u_xxx(t):
    return -2 * (2 + 6 * t)**-3

def numerical_solve(x_n, t_n, L, M, k = 1):
    h = 1 / (L - 1)
    tau = h * k
    u = [x[:] for x in [[0] * L] * M]
    u[0] = phi(x_n)
    psi_m = psi(t_n)
    u_x_m = u_x(t_n)
    u_xx_m = u_xx(t_n)
    u_xxx_m = u_xxx(t_n)
    for m in range(1, M):
        u[m][L - 1] = psi_m[m]
        u[m][L - 2] = psi_m[m] + u_x_m[m] * h + u_xx_m[m] * h**2 / 2 + u_xxx_m[m] * h**3 / 6
        u[m][L - 3] = psi_m[m] + u_x_m[m] * 2 * h + u_xx_m[m] * h**2 * 2 + u_xxx_m[m] * (h**3) * 4 / 3
    for m in range(M - 1):
        for l in reversed(range(L - 3)):
            u[m + 1][l] = u[m][l] + (2 * u[m][l + 3] - 9 * u[m][l + 2] + 18 * u[m][l + 1] - 11 * u[m][l]) * tau / h + (-u[m][l + 3] + 4 * u[m][l + 2] - 5 * u[m][l + 1] + 2 * u[m][l]) * 18 * (tau**2) / (h**2) + (u[m][l + 3] - 3 * u[m][l + 2] + 3 * u[m][l + 1] - u[m][l]) * 36 * (tau**3) / (h**3)
    return u

        Вывод решения

In [48]:
#находим аналитическое решение
x_rep = np.linspace(0, 1, 11)
u_analytical = analitical_solution(x_rep, 1)

#находим численное рашение
k = 0.45
L = 2561
M = int((L-1)/k)+1
x, t = grid(L, M)
u_n = numerical_solve(x, t, L, M, k)
u_numeric = np.array(u_n[-1][::int((len(u_n[0]) - 1) / 10)])

# запись данных в словарь и передча его в DataFrame для красоты
dat = {
    'x' : x_rep,
    'Analytical solve' : u_analytical,
    'Numeric solve' : u_numeric,
    'Error' : np.abs(u_analytical - u_numeric),
    'Max error' : np.max(np.abs(u_analytical - u_numeric))
}
df = pd.DataFrame(data=dat).T
df

,0,1,2,3,4,5,6,7,8,9,10
x,0.000000,0.100000,0.200000,0.300000,0.400000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Analytical solve,1.945910,1.960095,1.974081,1.987874,2.001480,2.014903,2.028148,2.041220,2.054124,2.066863,2.079442
Numeric solve,415.622945,6.896348,2.000966,1.991650,2.001434,2.014894,2.028140,2.041214,2.054120,2.066861,2.079442
Error,413.677035,4.936253,0.026885,0.003776,0.000046,0.000009,0.000008,0.000006,0.000004,0.000002,0.000000
Max error,413.677035,413.677035,413.677035,413.677035,413.677035,413.677035,413.677035,413.677035,413.677035,413.677035,413.677035
